<img src="images/imperial_logo.png" width="275" align="left">

<p style="text-align: right">
    **Created by** Amna Samar Askari<br>**Email:** asa713@ic.ac.uk<br><a>HTML Version (This will be a link)</a>
</p>
<br>

# Biot Savart Law

## Learning Objectives:
* Understanding the vectorial representation of quantities involved in calculating the magnetic field of a current loop 
* Being able to mathematically and computationally model these vectors in order to calculate the resultant magnetic field at a **point** an arbitrary distance away from the current loop 
* Understanding the influence of changing various parameters i.e. the size of the current loop or the position of the point on the resultant magnetic field

## Table of Contents: 

1. [The Biot Savart Law scenario](#one)
2. [Setting up the scenario with code](#two)
    1. [Initialisation](#initial)
    2. [Calculations](#calc)
3. [Visualising the results](#three)

# 1. The Biot Savart Law Scenario <a id="one"></a>

Referring to **7.4** in the Electricity & Magnetism, moving charges create a magnetic field (see explanation from Special Relativity) and we can use Ampere's law to find out the magnetic influence of a current carrying wire on a point in space. 

We used the Biot Savart Law for the more general case with an arbitrarily shaped current loop and an arbitrarily located point P. The situation can be represented by the figure below: 

<img src="images/Arbitrarycurrentloop.png" width="300" align="centre">

Equation 1 below (**Biot Savart Law**) relates the infitisemal magnetic field d**B** at the point **P** to the infitisemal current d**I** within the wire, which is consistent with Ampere's law: 

\\[ d\mathbf{B} = \frac{{\mu}_o}{4\pi} \cdot \frac{Id\mathbf{I} \times \mathbf{r}}{r^3} \\]

<p style="text-align: center">

<i>(For details on how we get this expression, see 7.4.4 in the notes) </i>
</p>
<br>
By the principle of superposition, the total magnetic field is at point **P** is given in Equation 2 by the integral: 

$$\mathbf{B} = \frac{{\mu}_oI}{4\pi}  \int_{C}\frac{d\mathbf{I} \times \mathbf{\hat{r}}}{r^2} $$

where $\mu_o$, the permeability of free space is a constant of value $4\pi$ x $10^{-7}$Tm A$^{-1}$,
<br>
*I* is the scalar magnitude of the current in the loop/wire, 
<br>
**r** is the displacement vector from the base of the d**I** vector to the point P, 
<br> 
r is the scalar magnitude of the vector **r** i.e. r = |**r**|, (<i>Note that this is why in the second equation, r is squared rather than cubed as the vector in the numerator is $\hat{r}$ which is the unit vector of r</i>)
<br>
C refers to the geometrical representation of the current loop, which we integrate over.

The figure below *(taken from 7.4.4 in the notes)* shows the equivalent geometrical representation for a circular current loop of radius a, with a point positioned on the axis of the current loop: 

<img src="images/Fig1.png" width="300" align="centre">



# 2. Setting up the scenario with code (Circular current loop) <a id="two"></a>


#### Our aim now is to recreate this system using Python so that we visualise the result of changing various parameters like the points position, the size of the current loop and which current element we pick.


## 2.A. Initialisation <a id="initial"></a>

We start by importing the required modules and ensuring that the notebook can contain Plotly (package used for visualisations) graphs:



In [6]:
import numpy as np #for mathematical operations and constants
import itertools #
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot #for visualisation 
import plotly.graph_objs as go #for visualisation
from scipy.integrate import quad #for integration

init_notebook_mode(connected=True) #to ensure that we can include live visualisations in the notebook

import sys 
#Mangle/Ronnie you will need to change this path to the server path when it's ready so that the Arrows.py module is imported 
sys.path.insert(0, "/Users/amnasaskari/Documents/Visualisations/Imperial-Visualizations/visuals_EM/BiotSavart/scripts")

Now, we can set up our variables and parameters. The comments in the code explain each variable in a little more detail and are enclosed with the '#' symbol

In [7]:
a = 0.35; #Radius of the circle

shape = "circle" #Shape of current loop. Default = Circle. 

perm = 4*np.pi*10e-7 #Permeability of free space 

current = 2.; #Scalar value of current or 'I'

P = np.array([0,0,1]); #The position of the point in 3D. Default = (0,0,1) i.e. the point is positioned
                       #on the axis of the current loop like in the diagram above. 

phi_scal = 0 #This variable determines the point at which the current element is positioned in the circle. Default = 0. 
             #It ranges from 0 to 2pi, as it goes around the circumference of the circle. You can 
             #imagine phi_scal to be the angle subtended by a vector which starts at the origin and ends at
             #the base of the current element at any arbitrary position.

B = np.empty((0, 3)) #Setting up the vector for the magnetic field as an empty array. It is a vector in 3D therefore
                     #it has dimensions (x,y,z)

#The following variables are for plotting the circular current loop, and therefore contain the cartesian coordinates of a circle
#of radius 'a', lying in the X-Y plane at elevation z=0. The default is using 50 points for the scatter plot, but one can use however many.
#A circle is plotted by sweeping the radius vector through 360 degrees/2pi...

theta_for_circle = np.linspace(0, 2 * np.pi, 50) #Again, this is the theta subtended by the r vector of the circle
xc = a*np.cos(theta_for_circle); # The x coordinates of the circle would be the horizontal component of the r vector
yc = a*np.sin(theta_for_circle); # The y coordinates of the circle would be the vertical component of the r vector
zc = np.zeros_like(xc); #The z coordinates of all points in the circle are 0. However, if you want to put it at a certain elevation, you can do so by changing this.

#The following variables are for plotting the axial line for the current loop:
z_axial = np.linspace(0,1.1,50)
x_axial = np.zeros_like(z_axial)
y_axial = np.zeros_like(z_axial)

## 2.A. Calculations <a id="calc"></a>

#### Defining functions: 

* The `create_vectors` function uses trigonometry to generate the **d**I and **r** vectors. `dI` resides in the X-Y plane at 0 elevation, and points radially outwards from the circumference of the circle. `r_vec` or **r** is the vector between the base of **d**I and the point `P`. 

* The `Biot_equation` function takes in the **d**I and **r** vectors generated by the `create_vectors` function above and implements them in Equation 1 to generate the **d**B vector. The permeability constant has been discluded as it would make **d**B a lot smaller than the other vectors in the plot, making it difficult to visualise. This vector is created solely for the purpose of visualisation and will not be passed over to integration calculations, as those are done over the entire current loop and this is just dependant on one current element. 
* To calculate the total magnetic field, we implement Equation (2) in the `calc_integrand` function to integrate over the entire current loop. The definitions of **d**I and **r** have been repeated in the function `calc_integrand`, using `phi` instead of `phi_scal`, as it is the variable that we need to integrate over. Hence, `result` is returned in terms of `phi`. We have specified `result[i]` as an element of a list as we need to calculate the integrand for 3 dimensions (x,y,z). 


In [8]:
def create_vectors(shape,phi_scal,P,a):
    if shape == "circle":
        dI= np.array([-a*np.sin(phi_scal),a*np.cos(phi_scal),0])
        r_vec = np.array([P[0]-a*np.cos(phi_scal),P[1]-a*np.sin(phi_scal),P[2]])
        
    return dI,r_vec 

def Biot_equation (current,perm,dI,r_vec):
    xprod = np.cross(dI,r_vec)
    dB = (current/(4*np.pi*(np.linalg.norm(r_vec)**3)))*xprod #should multiply current by perm 
    return dB

def calc_integrand(phi):
    a = 0.35;
    dI= np.array([-a*np.sin(phi),a*np.cos(phi),0])
    rvec = np.array([P[0]-a*np.cos(phi),P[1]-a*np.sin(phi),P[2]])
    result =(current/(4*np.pi*(np.linalg.norm(r_vec)**3)))* np.cross(dI,rvec)
    return result[i]

#### We now call these functions to generate 3D variables: 

* The code below calls the functions described above to generate the `dI`, `r_vec` and `dB` vectors which are used for visualisation 

* The `calc_integrand` function is passed through to scipy's integration function `quad` within a `for` loop that repeats the code three times for each dimension (x,y,z). The `result[i]` is the ith element of the magnetic field `B`. You can find the documentation for quad function <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html">here</a>, which will make it easy to follow and edit the code below. 

In [9]:
dI,r_vec = create_vectors("circle",phi_scal,P,a) 
dB= Biot_equation(current,perm,dI,r_vec) 

for j in range(3):
    i=j
    result_in_one_dimension = quad(calc_integrand,0,2*np.pi)[0]
    B = np.append(B,result_in_one_dimension,axis=None) 


# 3. Visualising the Results <a id="three"></a>

We use the package 'Plotly' to plot all the vectors that we have found. Each vector with its required plotting information comprises a `trace`. Plotly's `iplot` command (that actually generates the plot) takes in a `fig` dictionary that has an array of these traces as an argument. Each trace is a dictionary generated by Plotly's `go.Scatter3d` command and contains the following: 

* The nature of the plot. A scatter plot in 3D can either be a point, or a line. The vectors in this plot are plotted as lines and the point itself is plotted as a point.
* X,Y,Z coordinates: If the plot is a line, the X,Y,Z coordinates must be specified with a start and end point. The start point is the origin of the vector and the end point is the origin + the required component of the vector. 
* The color of the plot, specified in RGB coordinates. 

Feel free to adjust / remove any of the traces and see the result. Also, the plot is dynamic - feel free to drag / zoom / rotate!

### Note on Arrows: 

As we are plotting vectors and Plotly doesn't have inbuilt functionality for arrows, we need to specifically draw traces for arrows. Arrows are comprised of: 
* The **shaft**: This is the vector you are plotting 
* The **wings**: These are two vectors at a $45^{o}$ angle from the shaft that give the arrow its shape 

The `Arrows.py` module / python script in this directory contains a script for calculating wing traces for desired vectors. This generates the `traces` for both the **shaft** and **wings** using Plotly. For each of the vectors that we have found, namely:  **d**I, **d**B, **r**, and **B** - we generate 3D and 2D arrows accordingly. Other characeristic features of the diagram like the current loop, axis of the loop, position of the point and arrow indicating the direction of current are drawn using Plotly directly in this notebook. 


In [10]:
import Arrows as ar

dB_witharrow = ar.Arrow3D(cart_coords = dB,offset=P,width=6,color='rgb(50,100,200)',ratio=3,legend_status=True,label="dB",fraction=0.2)
dB_traces = dB_witharrow.data; 

B_witharrow = ar.Arrow3D(cart_coords = B,offset=P,width=6,color='rgb(255,160,75)',ratio=4,legend_status=True,label="B",fraction=0.2)
B_traces = B_witharrow.data; 

dI_witharrow = ar.Arrow2D(cart_coords = [dI[0],dI[1]],offset=[a*np.cos(phi_scal),a*np.sin(phi_scal)],width=6,color='rgb(250,20,0)',ratio=1,legend_status=True,label="dI")
dI_traces = dI_witharrow.data

R_witharrow = ar.Arrow3D(cart_coords = r_vec,offset=[a*np.cos(phi_scal),a*np.sin(phi_scal),0],width=6,color='rgb(0,220,0)',ratio=1,legend_status=True,label="R",fraction=0.1)
R_traces = R_witharrow.data; 


Circletrace = go.Scatter3d( 
                            x =  xc ,
                            y = yc,
                            z = zc,
                            mode = "markers",
                            marker = dict(symbol="circle",size=2.5,opacity=1,color= 'rgb(0,0,0)'),
                            showlegend =False
                          );


Circle_arrow = go.Scatter3d(
                            x = [a*np.cos(np.pi/4)-0.0025,a*np.cos(np.pi/4),a*np.cos(np.pi/4)+0.1], 
                            y = [a*np.sin(np.pi/4)-0.1,a*np.sin(np.pi/4),a*np.sin(np.pi/4)+0.0025], 
                            z = [0,0,0], 
                            mode = 'line',
                            line = dict(width = 4,color = 'rgb(0,0,0)'),
                            marker = dict(symbol="circle",size = 1, opacity =1,color= 'rgb(0,0,0)'),
                            showlegend = False,
                            );

Axial_line = go.Scatter3d(
                         x = x_axial, 
                         y = y_axial,
                         z = z_axial,
                         name = "axial",
                         mode = "markers",
                         marker = dict(symbol="circle",size = 1, opacity =1,color= 'rgb(0,0,0)'),
                         showlegend = False,
                         );

Pointtrace = go.Scatter3d(
                          x = [P[0]],
                          y= [P[1]],
                          z=[P[2]],
                          name = 'Point',
                          marker = dict(size=6, color= 'rgb(214,11,8)')
                         )


result= [dB_traces[0],dB_traces[1],B_traces[0],B_traces[1],dI_traces[0],dI_traces[1],R_traces[0],R_traces[1],Circletrace,Pointtrace,Axial_line,Circle_arrow]
fig = dict(data=result)

iplot(fig)
